In [38]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

## Import dataset

In [10]:
flights_df = pd.read_csv("data/flights.csv")

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
flights_df.index = pd.RangeIndex(len(flights_df.index))

In [12]:
flights_df.isnull().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

In [17]:
flights_agg = flights_df[['MONTH','DAY','DAY_OF_WEEK','AIRLINE','ORIGIN_AIRPORT',
                          'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','SCHEDULED_TIME',
                          'DISTANCE','SCHEDULED_ARRIVAL','DEPARTURE_DELAY']].copy()
flights_agg = flights_agg.dropna(axis=0, how = "any")

In [18]:
flights_agg.isnull().sum()

MONTH                  0
DAY                    0
DAY_OF_WEEK            0
AIRLINE                0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
SCHEDULED_TIME         0
DISTANCE               0
SCHEDULED_ARRIVAL      0
DEPARTURE_DELAY        0
dtype: int64

In [20]:
flights_agg['DELAY'] = 'NA'
flights_agg['DELAY'] = np.where(flights_agg['DEPARTURE_DELAY'] <= 0, 0, 1) #more efficient

## Data Balance

In [21]:
no_delay = (flights_agg['DELAY'] == 0).sum()
nobs = len(flights_agg['DELAY'])
no_delay_perc = float(no_delay)/nobs
delay_perc = 1 - no_delay_perc
print(no_delay_perc, delay_perc)

(0.6292264674895167, 0.3707735325104833)


## Undersampling

In [22]:
delay = nobs - no_delay
no_delay_indices = flights_agg[flights_agg.DELAY == 0].index
#undersamples no delays to equal same number of delays
np.random.seed(5)
random_indices = np.random.choice(no_delay_indices, delay, replace=False) 
no_delay_sample = flights_agg.loc[random_indices]

In [23]:
no_delay_sample[:10] #Check to make sure it was properly created

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,DELAY
2483768,6,6,6,EV,DFW,LEX,2020,140.0,785,2340,-2.0,0
4298638,9,25,5,MQ,ORD,IND,1142,73.0,177,1355,-2.0,0
1440480,4,3,5,WN,ATL,PHX,820,275.0,1587,955,-1.0,0
2073491,5,12,2,DL,DTW,LGA,1359,108.0,502,1547,-4.0,0
4999474,11,9,1,AS,BLI,SEA,640,49.0,93,729,-1.0,0
37497,1,3,6,DL,ATL,RSW,1225,102.0,515,1407,-4.0,0
38622,1,3,6,US,PHL,CLT,1340,107.0,449,1527,-4.0,0
4529081,10,9,5,WN,12889,14679,2135,60.0,258,2235,-2.0,0
3452835,8,3,1,OO,MSP,LAX,1530,247.0,1535,1737,-4.0,0
2298058,5,26,2,EV,ATL,ABE,1449,124.0,692,1653,-5.0,0


Create balanced sample

In [24]:
delay_sample = flights_agg[flights_agg.DELAY == 1] 
flights_agg_balanced = delay_sample.append(no_delay_sample)

In [25]:
n = int(len(flights_agg_balanced)*0.05)
flights_new_bal = flights_agg_balanced.sample(n, random_state = 33)

## Double check to make sure new sample is balanced

In [27]:
no_delay_bal = (flights_new_bal['DELAY'] == 0).sum()
no_delay_perc_bal = float(no_delay_bal)/n
delay_perc_bal = 1 - no_delay_perc_bal
print(no_delay_perc_bal, delay_perc_bal)

(0.5012208260217067, 0.4987791739782933)


## Compare original dataset with new sample

In [28]:
flights_agg.describe()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,DELAY
count,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06,5.732920e+06
mean,6.547214e+00,1.570853e+01,3.932390e+00,1.328963e+03,1.419380e+02,8.247620e+02,1.493375e+03,9.370097e+00,3.707735e-01
std,3.397111e+00,8.774777e+00,1.986191e+00,4.834725e+02,7.533436e+01,6.087944e+02,5.068416e+02,3.708078e+01,4.830120e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.800000e+01,2.100000e+01,1.000000e+00,-8.200000e+01,0.000000e+00
25%,4.000000e+00,8.000000e+00,2.000000e+00,9.160000e+02,8.600000e+01,3.730000e+02,1.110000e+03,-5.000000e+00,0.000000e+00
50%,7.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.230000e+02,6.500000e+02,1.520000e+03,-2.000000e+00,0.000000e+00
75%,9.000000e+00,2.300000e+01,6.000000e+00,1.730000e+03,1.740000e+02,1.065000e+03,1.917000e+03,7.000000e+00,1.000000e+00
max,1.200000e+01,3.100000e+01,7.000000e+00,2.359000e+03,7.180000e+02,4.983000e+03,2.400000e+03,1.988000e+03,1.000000e+00


In [29]:
flights_new_bal.describe() #matches

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,DELAY
count,212561.000000,212561.000000,212561.000000,212561.000000,212561.000000,212561.000000,212561.000000,212561.000000,212561.000000
mean,6.510503,15.694897,3.934753,1354.500934,143.242053,835.669074,1516.261059,14.038803,0.498779
std,3.393190,8.772973,1.988450,480.486667,75.822668,612.485407,508.504563,41.261328,0.500000
min,1.000000,1.000000,1.000000,3.000000,18.000000,31.000000,1.000000,-45.000000,0.000000
25%,4.000000,8.000000,2.000000,940.000000,86.000000,383.000000,1125.000000,-4.000000,0.000000
50%,7.000000,16.000000,4.000000,1346.000000,125.000000,667.000000,1545.000000,0.000000,0.000000
75%,9.000000,23.000000,6.000000,1741.000000,175.000000,1075.000000,1930.000000,14.000000,1.000000
max,12.000000,31.000000,7.000000,2359.000000,718.000000,4983.000000,2359.000000,1367.000000,1.000000


Build Airport dictionary to replace incomplete IATA airport codes for October

In [30]:
airport_table = pd.read_csv("data/airport_codes_clean.csv")

In [31]:
oct_flights = flights_new_bal[flights_new_bal.MONTH == 10] 
flights_new_bal = flights_new_bal[~flights_new_bal["MONTH"].isin([10])]

In [32]:
oct_flights["ORIGIN_AIRPORT"] = oct_flights["ORIGIN_AIRPORT"].apply(pd.to_numeric)
oct_flights["ORIGIN_AIRPORT"].replace(list(airport_table['Code']), 
                                                                list(airport_table['Airport Code']), inplace=True)
flights_new_bal = flights_new_bal.append(oct_flights)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Give unique airport IATA code unique integer and for airlines

In [33]:
unique_IATA = flights_new_bal['ORIGIN_AIRPORT'].unique()
unique_IATA_id = [x for x in range(1, len(unique_IATA) + 1)]
flights_new_bal['ORIGIN_AIRPORT'].replace(unique_IATA, unique_IATA_id, inplace = True)

In [34]:
unique_airline = flights_new_bal['AIRLINE'].unique()
unique_airline_id = [x for x in range(1, len(unique_airline) + 1)]
flights_new_bal['AIRLINE'].replace(unique_airline, unique_airline_id, inplace = True)

Visualization

In [36]:
month_delays = flights_new_bal[flights_new_bal['DELAY'] == 1]
ggp.ggplot(month_delays, ggp.aes(x = 'MONTH')) + ggp.geom_bar(fill = '#173265') +\
ggp.labs(title="Flight Delays per Month", x = "Month", y = "Flight Delays") 

NameError: name 'ggp' is not defined

In [39]:
correl = mpl.matshow(flights_new_bal.corr(), cmap = mpl.cm.Greens)
mpl.show(correl)

NameError: name 'mpl' is not defined

## Initializing SQL Dataframe

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

In [62]:
#flight_df = sqlCtx.createDataFrame(flights_new_bal)
flight_df2 = sqlCtx.createDataFrame(flights_new_bal, ['features'])

In [66]:
data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = sqlCtx.createDataFrame(data, ["features"])

In [67]:
flight_df2.show(5)
flight_df.show(5)
df.show(5)

+--------+---+---+-----+-----+------+----+---+----+---+
|features| _2| _3|   _4|   _5|    _6|  _7| _8|  _9|_10|
+--------+---+---+-----+-----+------+----+---+----+---+
|       5|  5|  1|146.0|128.0|1047.0| 7.0|  1|1040|  L|
|       5|  2|  2|117.0|104.0|1639.0|34.0|  1|1605|  L|
|       3|  3|  3| 93.0| 74.0|1655.0|-5.0|  2|1700|  S|
|      11|  3|  4| 78.0| 62.0|1831.0|36.0|  3|1755|  L|
|       8|  2|  5|115.0| 91.0|1102.0|-6.0|  4|1108|  S|
+--------+---+---+-----+-----+------+----+---+----+---+
only showing top 5 rows

+-----+-----------+--------------+------------+--------+--------------+---------------+-------+-------------------+-----+
|MONTH|DAY_OF_WEEK|ORIGIN_AIRPORT|ELAPSED_TIME|AIR_TIME|DEPARTURE_TIME|DEPARTURE_DELAY|AIRLINE|SCHEDULED_DEPARTURE|Delay|
+-----+-----------+--------------+------------+--------+--------------+---------------+-------+-------------------+-----+
|    5|          5|             1|       146.0|   128.0|        1047.0|            7.0|      1|          

## PCA

In [53]:
from pyspark.ml.feature import PCA
from pyspark.mllib.linalg import Vectors

In [56]:
pca = PCA(k = 5, inputCol = 'features', outputCol = 'pcaFeatures')

In [68]:
pca_model = pca.fit(lp)

NameError: name 'lp' is not defined

## Logistic Regression

In [57]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.stat import Correlation

In [72]:
flight_df.select(['DELAY', 'MONTH','ORIGIN_AIRPORT', 'DAY_OF_WEEK', 'ELAPSED_TIME', 'DEPARTURE_TIME', 
                  'AIR_TIME', 'AIRLINE', 'SCHEDULED_DEPARTURE']).show(3)

+-----+-----+--------------+-----------+------------+--------------+--------+-------+-------------------+
|Delay|MONTH|ORIGIN_AIRPORT|DAY_OF_WEEK|ELAPSED_TIME|DEPARTURE_TIME|AIR_TIME|AIRLINE|SCHEDULED_DEPARTURE|
+-----+-----+--------------+-----------+------------+--------------+--------+-------+-------------------+
|    L|    5|             1|          5|       146.0|        1047.0|   128.0|      1|               1040|
|    L|    5|             2|          2|       117.0|        1639.0|   104.0|      1|               1605|
|    S|    3|             3|          3|        93.0|        1655.0|    74.0|      2|               1700|
+-----+-----+--------------+-----------+------------+--------------+--------+-------+-------------------+
only showing top 3 rows



In [73]:
assembler = VectorAssembler(inputCols=['MONTH', 'DAY_OF_WEEK', 'ELAPSED_TIME', 'AIR_TIME', 'SCHEDULED_DEPARTURE', 
                                       'AIR_TIME'], outputCol="features")

NameError: name 'VectorAssembler' is not defined

In [74]:
transformed = assembler.transform(flight_df)

NameError: name 'assembler' is not defined

In [75]:
transformed.select(['DELAY', 'features']).show(5)

NameError: name 'transformed' is not defined

In [70]:
dataRDD = transformed.select(['DELAY','features']).rdd.map(tuple)

NameError: name 'transformed' is not defined

In [69]:
lp = dataRDD.map(lambda row : (0 if row[0] == 0 else 1, Vectors.dense(row[1])))    \
            .map(lambda row : LabeledPoint(row[0], row[1]))

NameError: name 'dataRDD' is not defined

In [32]:
lp.take(11)

[LabeledPoint(1.0, [5.0,5.0,146.0,128.0,1040.0,128.0]),
 LabeledPoint(1.0, [5.0,2.0,117.0,104.0,1605.0,104.0]),
 LabeledPoint(0.0, [3.0,3.0,93.0,74.0,1700.0,74.0]),
 LabeledPoint(1.0, [11.0,3.0,78.0,62.0,1755.0,62.0]),
 LabeledPoint(0.0, [8.0,2.0,115.0,91.0,1108.0,91.0]),
 LabeledPoint(0.0, [9.0,5.0,138.0,123.0,2030.0,123.0]),
 LabeledPoint(1.0, [6.0,5.0,170.0,144.0,1635.0,144.0]),
 LabeledPoint(0.0, [12.0,1.0,99.0,80.0,1325.0,80.0]),
 LabeledPoint(1.0, [1.0,2.0,57.0,43.0,2115.0,43.0]),
 LabeledPoint(0.0, [9.0,4.0,127.0,77.0,1417.0,77.0]),
 LabeledPoint(0.0, [5.0,7.0,53.0,38.0,1330.0,38.0])]

In [33]:
split = lp.randomSplit([0.8, 0.2], 314)
training = split[0]
test = split[1]

In [34]:
LR_model = LogisticRegressionWithLBFGS.train(training)

In [35]:
LR_LAP = test.map(lambda lp: (float(LR_model.predict(lp.features)), lp.label))

In [36]:
LR_acc = 1.0 * LR_LAP.filter(lambda x:x[0] == x[1]).count()/test.count()
print(LR_acc)

0.5746268656716418


In [60]:
Correlation.corr(transformed, "features")

+--------------------+
|   pearson(features)|
+--------------------+
|1.0              ...|
+--------------------+



## Random Forest

In [37]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier

In [73]:
RF_model = RandomForest.trainClassifier(training, numClasses = 2,
                                       categoricalFeaturesInfo = {}, 
                                       numTrees = 400, featureSubsetStrategy = "auto", 
                                       impurity = 'gini', maxDepth = 4, maxBins = 32)

In [74]:
RF_pred = RF_model.predict(test.map(lambda x: x.features))

In [75]:
RF_LAP = test.map(lambda lp: lp.label).zip(RF_pred)

In [76]:
RF_testErr = RF_LAP.filter(lambda x: x[0] == x[1]).count()/float(test.count())
print(RF_testErr)

0.5962568111821843


## Decision Tree

In [46]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

In [47]:
DT_model = DecisionTree.trainClassifier(training, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

In [48]:
DT_pred = DT_model.predict(test.map(lambda x: x.features))

In [49]:
DT_LAP = test.map(lambda lp: lp.label).zip(DT_pred)

In [50]:
DT_LAP = test.map(lambda lp: lp.label).zip(DT_pred)
DT_testErr = DT_LAP.filter(lambda x: x[0] == x[1]).count() / float(test.count())
print(DT_testErr)

0.6005685856432125


## SVM

In [51]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

In [52]:
SVM_model = SVMWithSGD.train(training, iterations = 1000)

In [53]:
SVM_LAP = test.map(lambda x: (float(SVM_model.predict(x.features)), x.label))

In [54]:
SVM_testErr = SVM_LAP.filter(lambda x: x[0] == x[1]).count()/float(test.count())
print(SVM_testErr)

0.5027007818052595


## K-Fold Cross Validation

In [60]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder